# Fuel Prices Analysis (United Kingdom Data)

## Research Goals

- Identify how fuel prices fluctuate over time and determine key patterns.
- Analyze the impact of external factors such as supply chain disruptions, oil prices, VAT, inflation, and geopolitical events on fuel prices.
- Compare fuel price trends across different time periods to understand long-term changes.
- Use machine learning models to predict future fuel price movements based on historical data.
- Store and process data efficiently using **AWS S3** and **Snowflake**, ensuring scalability and reliability.
- Present findings through interactive **Power BI** dashboards and **Plotly Dash** visualizations for clear and actionable insights.
- Implement a **deployment mode** for predictive models, allowing real-time or batch forecasting of fuel prices.

## 8. Deep Analysis: Next Steps

### **Steps**
1. **Data Validation & Cleaning**  
   - Ensure all data is properly formatted and aligned for further analysis.
   - Handle missing values and inconsistencies.

In [4]:
pip install snowflake-connector-python

Note: you may need to restart the kernel to use updated packages.


#### 8.1 Snowflake connection

#### 8.2 Data Uploading

In [7]:


# Create cursor to execute queries
cursor = conn.cursor()


In [8]:
query = "SELECT * FROM fuel.fuel"
cursor.execute(query)

# Fetch results and convert to pandas DataFrame
import pandas as pd
df_fuel = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])

# Display the DataFrame
print(df_fuel.head())


         DATE  ULSP_PRICE  ULSD_PRICE  ULSP_DUTY  ULSD_DUTY
0  2019-01-07      1.2027      1.3033     0.5795     0.5795
1  2019-01-14      1.1953      1.2947     0.5795     0.5795
2  2019-01-21      1.1912      1.2892     0.5795     0.5795
3  2019-01-28      1.1929      1.2910     0.5795     0.5795
4  2019-02-04      1.1913      1.2913     0.5795     0.5795


In [9]:
query = "SELECT * FROM oil.oil"  # You can adjust the query as needed
cursor.execute(query)

# Fetch results and convert to pandas DataFrame
import pandas as pd
df_oil = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])

# Display the DataFrame
print(df_oil.head())

         DATE  PRICE
0  2019-01-01  59.41
1  2019-02-01  63.96
2  2019-03-01  66.14
3  2019-04-01  71.23
4  2019-05-01  71.32


In [10]:
query = "SELECT * FROM currency.currency"
cursor.execute(query)

# Fetch results and convert to pandas DataFrame
import pandas as pd
df_currency = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])

# Display the DataFrame
print(df_currency.head())

         DATE   PRICE    OPEN    HIGH     LOW  CHANGE_PERCENTAGE
0  2024-12-31  1.2511  1.2548  1.2571  1.2504            -0.0032
1  2024-12-30  1.2551  1.2571  1.2608  1.2505            -0.0022
2  2024-12-27  1.2579  1.2523  1.2594  1.2503             0.0043
3  2024-12-26  1.2525  1.2552  1.2553  1.2500            -0.0022
4  2024-12-25  1.2553  1.2543  1.2560  1.2526             0.0009


In [11]:
query = "SELECT * FROM inflation.inflation LIMIT 10"  # You can adjust the query as needed
cursor.execute(query)

# Fetch results and convert to pandas DataFrame
import pandas as pd
df_inflation = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])

# Display the DataFrame
print(df_inflation.head())

         DATE  INFLATION_RATE
0  2019-01-01             1.8
1  2019-02-01             1.8
2  2019-03-01             1.8
3  2019-04-01             2.0
4  2019-05-01             1.9


#### 8.3 Creating Base Date DataFrame

##### **Objective**
We need a complete date range from **2019-01-01 to 2024-12-31** to ensure that all datasets align properly.  
Since different datasets have different date frequencies, this will serve as the foundation for merging.

##### **Steps**
1. Create a DataFrame (`df_base`) with all dates from **2019-01-01 to 2024-12-31**.
2. Convert `DATE` to string format to match other datasets.


In [17]:
import pandas as pd

# Create a date range from 2019-01-01 to 2024-12-31
date_range = pd.date_range(start="2019-01-01", end="2024-12-31", freq="D")

# Create a DataFrame with this date range
df_base = pd.DataFrame(date_range, columns=["DATE"])

# Convert DATE to datetime format (even though it's already in datetime format after creation)
df_base["DATE"] = pd.to_datetime(df_base["DATE"])

# Verify the data type
print(df_base.dtypes)
print(df_base.head())

DATE    datetime64[ns]
dtype: object
        DATE
0 2019-01-01
1 2019-01-02
2 2019-01-03
3 2019-01-04
4 2019-01-05


In [18]:
df_base.dtypes

DATE    datetime64[ns]
dtype: object

#### 8.4: Grouping and Structuring Columns

In [21]:
# Convert DATE column to datetime format for consistency
df_fuel["DATE"] = pd.to_datetime(df_fuel["DATE"])
df_base["DATE"] = pd.to_datetime(df_base["DATE"])

# Check for duplicate dates
duplicates = df_fuel[df_fuel.duplicated(subset=["DATE"], keep=False)]
print("Duplicates in df_fuel:", duplicates)

Duplicates in df_fuel: Empty DataFrame
Columns: [DATE, ULSP_PRICE, ULSD_PRICE, ULSP_DUTY, ULSD_DUTY]
Index: []


In [23]:
df_fuel = df_fuel.groupby("DATE", as_index=False).mean()

In [24]:
# Merge df_fuel with df_base using a left join to retain all dates
df_merged = df_base.merge(df_fuel, on="DATE", how="left")

# Check the first few rows to verify
print(df_merged.head())

# Check for missing values after merging
print(df_merged.isnull().sum())


        DATE  ULSP_PRICE  ULSD_PRICE  ULSP_DUTY  ULSD_DUTY
0 2019-01-01         NaN         NaN        NaN        NaN
1 2019-01-02         NaN         NaN        NaN        NaN
2 2019-01-03         NaN         NaN        NaN        NaN
3 2019-01-04         NaN         NaN        NaN        NaN
4 2019-01-05         NaN         NaN        NaN        NaN
DATE             0
ULSP_PRICE    1879
ULSD_PRICE    1879
ULSP_DUTY     1879
ULSD_DUTY     1879
dtype: int64


In [27]:
print(df_merged.head(20))

         DATE  ULSP_PRICE  ULSD_PRICE  ULSP_DUTY  ULSD_DUTY
0  2019-01-01         NaN         NaN        NaN        NaN
1  2019-01-02         NaN         NaN        NaN        NaN
2  2019-01-03         NaN         NaN        NaN        NaN
3  2019-01-04         NaN         NaN        NaN        NaN
4  2019-01-05         NaN         NaN        NaN        NaN
5  2019-01-06         NaN         NaN        NaN        NaN
6  2019-01-07      1.2027      1.3033     0.5795     0.5795
7  2019-01-08         NaN         NaN        NaN        NaN
8  2019-01-09         NaN         NaN        NaN        NaN
9  2019-01-10         NaN         NaN        NaN        NaN
10 2019-01-11         NaN         NaN        NaN        NaN
11 2019-01-12         NaN         NaN        NaN        NaN
12 2019-01-13         NaN         NaN        NaN        NaN
13 2019-01-14      1.1953      1.2947     0.5795     0.5795
14 2019-01-15         NaN         NaN        NaN        NaN
15 2019-01-16         NaN         NaN   

---

#### 8.5 Merging Oil Data

In [28]:
# Ensure DATE is in datetime format for consistency
df_oil["DATE"] = pd.to_datetime(df_oil["DATE"])

# Merge df_oil with df_merged using a left join
df_merged = df_merged.merge(df_oil, on="DATE", how="left")

# Display first 20 rows after merging df_oil
print(df_merged.head(20))

         DATE  ULSP_PRICE  ULSD_PRICE  ULSP_DUTY  ULSD_DUTY  PRICE
0  2019-01-01         NaN         NaN        NaN        NaN  59.41
1  2019-01-02         NaN         NaN        NaN        NaN    NaN
2  2019-01-03         NaN         NaN        NaN        NaN    NaN
3  2019-01-04         NaN         NaN        NaN        NaN    NaN
4  2019-01-05         NaN         NaN        NaN        NaN    NaN
5  2019-01-06         NaN         NaN        NaN        NaN    NaN
6  2019-01-07      1.2027      1.3033     0.5795     0.5795    NaN
7  2019-01-08         NaN         NaN        NaN        NaN    NaN
8  2019-01-09         NaN         NaN        NaN        NaN    NaN
9  2019-01-10         NaN         NaN        NaN        NaN    NaN
10 2019-01-11         NaN         NaN        NaN        NaN    NaN
11 2019-01-12         NaN         NaN        NaN        NaN    NaN
12 2019-01-13         NaN         NaN        NaN        NaN    NaN
13 2019-01-14      1.1953      1.2947     0.5795     0.5795   

---

#### 8.6 Merging Currency Data

In [30]:
# Ensure DATE is in datetime format for consistency
df_currency["DATE"] = pd.to_datetime(df_currency["DATE"])

# Merge df_currency with df_merged using a left join
df_merged = df_merged.merge(df_currency, on="DATE", how="left")

# Display first 20 rows after merging df_currency
print(df_merged.head(20))

         DATE  ULSP_PRICE  ULSD_PRICE  ULSP_DUTY  ULSD_DUTY  PRICE_x  PRICE_y  \
0  2019-01-01         NaN         NaN        NaN        NaN    59.41   1.2751   
1  2019-01-02         NaN         NaN        NaN        NaN      NaN   1.2607   
2  2019-01-03         NaN         NaN        NaN        NaN      NaN   1.2631   
3  2019-01-04         NaN         NaN        NaN        NaN      NaN   1.2726   
4  2019-01-05         NaN         NaN        NaN        NaN      NaN      NaN   
5  2019-01-06         NaN         NaN        NaN        NaN      NaN      NaN   
6  2019-01-07      1.2027      1.3033     0.5795     0.5795      NaN   1.2778   
7  2019-01-08         NaN         NaN        NaN        NaN      NaN   1.2717   
8  2019-01-09         NaN         NaN        NaN        NaN      NaN   1.2788   
9  2019-01-10         NaN         NaN        NaN        NaN      NaN   1.2747   
10 2019-01-11         NaN         NaN        NaN        NaN      NaN   1.2841   
11 2019-01-12         NaN   

---

#### 8.7 Drop Unnecessary Columns from Currency

In [31]:
# Keep only the PRICE column from df_currency
df_merged = df_merged.drop(columns=["OPEN", "HIGH", "LOW", "CHANGE_PERCENTAGE"])

# Rename PRICE_y to PRICE_CURRENCY for clarity
df_merged = df_merged.rename(columns={"PRICE_y": "PRICE_CURRENCY", "PRICE_x": "PRICE_OIL"})

# Display first 20 rows to verify
print(df_merged.head(20))

         DATE  ULSP_PRICE  ULSD_PRICE  ULSP_DUTY  ULSD_DUTY  PRICE_OIL  \
0  2019-01-01         NaN         NaN        NaN        NaN      59.41   
1  2019-01-02         NaN         NaN        NaN        NaN        NaN   
2  2019-01-03         NaN         NaN        NaN        NaN        NaN   
3  2019-01-04         NaN         NaN        NaN        NaN        NaN   
4  2019-01-05         NaN         NaN        NaN        NaN        NaN   
5  2019-01-06         NaN         NaN        NaN        NaN        NaN   
6  2019-01-07      1.2027      1.3033     0.5795     0.5795        NaN   
7  2019-01-08         NaN         NaN        NaN        NaN        NaN   
8  2019-01-09         NaN         NaN        NaN        NaN        NaN   
9  2019-01-10         NaN         NaN        NaN        NaN        NaN   
10 2019-01-11         NaN         NaN        NaN        NaN        NaN   
11 2019-01-12         NaN         NaN        NaN        NaN        NaN   
12 2019-01-13         NaN         NaN 

---

#### 8.8 Merging Inflation Data

In [32]:
# Ensure DATE is in datetime format for consistency
df_inflation["DATE"] = pd.to_datetime(df_inflation["DATE"])

# Merge df_inflation with df_merged using a left join
df_merged = df_merged.merge(df_inflation, on="DATE", how="left")

# Display first 20 rows after merging df_inflation
print(df_merged.head(20))

         DATE  ULSP_PRICE  ULSD_PRICE  ULSP_DUTY  ULSD_DUTY  PRICE_OIL  \
0  2019-01-01         NaN         NaN        NaN        NaN      59.41   
1  2019-01-02         NaN         NaN        NaN        NaN        NaN   
2  2019-01-03         NaN         NaN        NaN        NaN        NaN   
3  2019-01-04         NaN         NaN        NaN        NaN        NaN   
4  2019-01-05         NaN         NaN        NaN        NaN        NaN   
5  2019-01-06         NaN         NaN        NaN        NaN        NaN   
6  2019-01-07      1.2027      1.3033     0.5795     0.5795        NaN   
7  2019-01-08         NaN         NaN        NaN        NaN        NaN   
8  2019-01-09         NaN         NaN        NaN        NaN        NaN   
9  2019-01-10         NaN         NaN        NaN        NaN        NaN   
10 2019-01-11         NaN         NaN        NaN        NaN        NaN   
11 2019-01-12         NaN         NaN        NaN        NaN        NaN   
12 2019-01-13         NaN         NaN 

In [33]:
# Save the merged DataFrame to CSV before cleaning
df_merged.to_csv("merged_data_before_cleaning.csv", index=False)

# Verify that the file was saved correctly
print("File saved: merged_data_before_cleaning.csv")

File saved: merged_data_before_cleaning.csv


---

####  Step 8.9: Filling Missing Values
To ensure smooth analysis and avoid gaps, we use **forward fill (`ffill`)**, which replaces `NaN` values with the **last known** value.

#### **Filling Strategy**
1. **Forward fill (`ffill`)** – Each missing value is replaced by the previous available value.
2. **If the first rows are missing values**, they remain `NaN` (we may apply `bfill` later if needed)

In [34]:
# Fill missing values using forward fill (ffill)
df_merged.fillna(method="ffill", inplace=True)

# Display first 20 rows to verify
print(df_merged.head(20))

         DATE  ULSP_PRICE  ULSD_PRICE  ULSP_DUTY  ULSD_DUTY  PRICE_OIL  \
0  2019-01-01         NaN         NaN        NaN        NaN      59.41   
1  2019-01-02         NaN         NaN        NaN        NaN      59.41   
2  2019-01-03         NaN         NaN        NaN        NaN      59.41   
3  2019-01-04         NaN         NaN        NaN        NaN      59.41   
4  2019-01-05         NaN         NaN        NaN        NaN      59.41   
5  2019-01-06         NaN         NaN        NaN        NaN      59.41   
6  2019-01-07      1.2027      1.3033     0.5795     0.5795      59.41   
7  2019-01-08      1.2027      1.3033     0.5795     0.5795      59.41   
8  2019-01-09      1.2027      1.3033     0.5795     0.5795      59.41   
9  2019-01-10      1.2027      1.3033     0.5795     0.5795      59.41   
10 2019-01-11      1.2027      1.3033     0.5795     0.5795      59.41   
11 2019-01-12      1.2027      1.3033     0.5795     0.5795      59.41   
12 2019-01-13      1.2027      1.3033 

C:\Users\Zana\AppData\Local\Temp\ipykernel_1508\1667416341.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged.fillna(method="ffill", inplace=True)


In [35]:
# Use forward fill correctly
df_merged.ffill(inplace=True)

# Display first 20 rows to verify
print(df_merged.head(20))

         DATE  ULSP_PRICE  ULSD_PRICE  ULSP_DUTY  ULSD_DUTY  PRICE_OIL  \
0  2019-01-01         NaN         NaN        NaN        NaN      59.41   
1  2019-01-02         NaN         NaN        NaN        NaN      59.41   
2  2019-01-03         NaN         NaN        NaN        NaN      59.41   
3  2019-01-04         NaN         NaN        NaN        NaN      59.41   
4  2019-01-05         NaN         NaN        NaN        NaN      59.41   
5  2019-01-06         NaN         NaN        NaN        NaN      59.41   
6  2019-01-07      1.2027      1.3033     0.5795     0.5795      59.41   
7  2019-01-08      1.2027      1.3033     0.5795     0.5795      59.41   
8  2019-01-09      1.2027      1.3033     0.5795     0.5795      59.41   
9  2019-01-10      1.2027      1.3033     0.5795     0.5795      59.41   
10 2019-01-11      1.2027      1.3033     0.5795     0.5795      59.41   
11 2019-01-12      1.2027      1.3033     0.5795     0.5795      59.41   
12 2019-01-13      1.2027      1.3033 

---

#### 8.10 Apply Backward Fill

In [36]:
# Apply backward fill (bfill) to fill remaining NaN values
df_merged.bfill(inplace=True)

# Display first 20 rows to verify
print(df_merged.head(20))

         DATE  ULSP_PRICE  ULSD_PRICE  ULSP_DUTY  ULSD_DUTY  PRICE_OIL  \
0  2019-01-01      1.2027      1.3033     0.5795     0.5795      59.41   
1  2019-01-02      1.2027      1.3033     0.5795     0.5795      59.41   
2  2019-01-03      1.2027      1.3033     0.5795     0.5795      59.41   
3  2019-01-04      1.2027      1.3033     0.5795     0.5795      59.41   
4  2019-01-05      1.2027      1.3033     0.5795     0.5795      59.41   
5  2019-01-06      1.2027      1.3033     0.5795     0.5795      59.41   
6  2019-01-07      1.2027      1.3033     0.5795     0.5795      59.41   
7  2019-01-08      1.2027      1.3033     0.5795     0.5795      59.41   
8  2019-01-09      1.2027      1.3033     0.5795     0.5795      59.41   
9  2019-01-10      1.2027      1.3033     0.5795     0.5795      59.41   
10 2019-01-11      1.2027      1.3033     0.5795     0.5795      59.41   
11 2019-01-12      1.2027      1.3033     0.5795     0.5795      59.41   
12 2019-01-13      1.2027      1.3033 

In [37]:
# Save the cleaned dataset to CSV
df_merged.to_csv("final_cleaned_data.csv", index=False)

# Confirm that the file was saved
print("File saved: final_cleaned_data.csv")

File saved: final_cleaned_data.csv


### Completed Steps: Data Cleaning & Preparation

#### 1. Data Collection & Merging
- Extracted **fuel prices, oil prices, currency exchange rates, and inflation data**.
- Merged all datasets into a **single DataFrame** with a complete date range.

#### 2. Handling Missing Values
- Applied **forward fill (ffill)** to propagate last known values.
- Applied **backward fill (bfill)** to ensure all missing values were replaced.

#### 3. Column Standardization & Renaming
- Renamed columns for clarity:
  - `"PRICE_x"` → `"PRICE_OIL"`
  - `"PRICE_y"` → `"PRICE_CURRENCY"`
- Dropped unnecessary columns (e.g., `"OPEN"`, `"HIGH"`, `"LOW"`, `"CHANGE_PERCENTAGE"` from currency data).

#### 4. Data Cleaning Validation
- Ensured **date consistency** across all datasets.
- Verified **no duplicate values**.
- Checked for **remaining NaN values** after filling.

#### 5. Data Storage
- Saved the final cleaned dataset as **`final_cleaned_data.csv`**.
- Ready for **Exploratory Data Analysis (EDA)**. 
